In [3]:
%matplotlib inline

In [8]:
import numpy as np
import gdist
from __future__ import division
import scipy.spatial as spatial

In [5]:
# -*- coding: utf-8 -*-
'''
Functions to read and write vtk files
------------------------------------
* read takes vtk file and returns vertex and face array
* write takes vertex and faces array and optional comment and returns vtk file
* reading and writing of texture not supported yet

TO DO: add reading comments
'''

def read_vtk(file):
    import pandas as pd
    import numpy as np
    # read full file while dropping empty lines 
    vtk_df=pd.read_csv(file, header=None)
    vtk_df=vtk_df.dropna()
    # extract number of vertices and faces
    number_vertices=int(vtk_df[vtk_df[0].str.contains('POINTS')][0].iloc[0].split()[1])
    number_faces=int(vtk_df[vtk_df[0].str.contains('POLYGONS')][0].iloc[0].split()[1])
    # read vertices into df and array
    start_vertices= (vtk_df[vtk_df[0].str.contains('POINTS')].index.tolist()[0])+1
    vertex_df=pd.read_csv(file, skiprows=range(start_vertices), nrows=number_vertices, sep='\s*', header=None)
    if np.array(vertex_df).shape[1]==3:
        vertex_array=np.array(vertex_df)
    # sometimes the vtk format is weird with 9 indices per line, then it has to be reshaped
    elif np.array(vertex_df).shape[1]==9:
        vertex_df=pd.read_csv(file, skiprows=range(start_vertices), nrows=int(number_vertices/3+1), sep='\s*', header=None)
        vertex_array=np.array(vertex_df.iloc[0:1,0:3])
        vertex_array=np.append(vertex_array, vertex_df.iloc[0:1,3:6], axis=0)
        vertex_array=np.append(vertex_array, vertex_df.iloc[0:1,6:9], axis=0)
        for row in range(1,int(number_vertices/3+1)):
            for col in [0,3,6]:
                vertex_array=np.append(vertex_array, array(vertex_df.iloc[row:(row+1),col:(col+3)]),axis=0) 
        # strip rows containing nans
        vertex_array=vertex_array[ ~np.isnan(vertex_array) ].reshape(number_vertices,3)
    else:
        print "vertex indices out of shape"
    # read faces into df and array
    start_faces= (vtk_df[vtk_df[0].str.contains('POLYGONS')].index.tolist()[0])+1
    face_df=pd.read_csv(file, skiprows=range(start_faces), nrows=number_faces, sep='\s*', header=None)
    face_array=np.array(face_df.iloc[:,1:4])
    # read data into df and array if exists
    if vtk_df[vtk_df[0].str.contains('POINT_DATA')].index.tolist()!=[]:
        start_data=(vtk_df[vtk_df[0].str.contains('POINT_DATA')].index.tolist()[0])+3
        number_data = number_vertices
        data_df=pd.read_csv(file, skiprows=range(start_data), nrows=number_data, sep='\s*', header=None)
        data_array=np.array(data_df)
    else:
        data_array = np.empty(0)
    
    return vertex_array, face_array, data_array



def write_vtk(filename, vertices, faces, data=None, comment=None):
    import pandas as pd
    import numpy as np
    # infer number of vertices and faces
    number_vertices=vertices.shape[0]
    number_faces=faces.shape[0]
    if data is not None:
        number_data=data.shape[0]
    # make header and subheader dataframe
    header=['# vtk DataFile Version 3.0',
            '%s'%comment,
            'ASCII',
            'DATASET POLYDATA',
            'POINTS %i float'%number_vertices
             ]
    header_df=pd.DataFrame(header)
    sub_header=['POLYGONS %i %i'%(number_faces, 4*number_faces)]
    sub_header_df=pd.DataFrame(sub_header)    
    # make dataframe from vertices
    vertex_df=pd.DataFrame(vertices)
    # make dataframe from faces, appending first row of 3's (indicating the polygons are triangles)
    triangles=np.reshape(3*(np.ones(number_faces)), (number_faces,1))
    triangles=triangles.astype(int)
    faces=faces.astype(int)
    faces_df=pd.DataFrame(np.concatenate((triangles,faces),axis=1))
    # write dfs to csv
    header_df.to_csv(filename, header=None, index=False)
    with open(filename, 'a') as f:
        vertex_df.to_csv(f, header=False, index=False, float_format='%.3f', sep=' ')
    with open(filename, 'a') as f:
        sub_header_df.to_csv(f, header=False, index=False)
    with open(filename, 'a') as f:
        faces_df.to_csv(f, header=False, index=False, float_format='%.0f', sep=' ')
    # if there is data append second subheader and data
    if data!=None:
        datapoints=data.shape[1]
        sub_header2=['POINT_DATA %i'%(number_data),
                     'SCALARS EmbedVertex float %i'%(datapoints),
                     'LOOKUP_TABLE default']
        sub_header_df2=pd.DataFrame(sub_header2)
        data_df=pd.DataFrame(data)
        with open(filename, 'a') as f:
            sub_header_df2.to_csv(f, header=False, index=False)
        with open(filename, 'a') as f:
            data_df.to_csv(f, header=False, index=False, float_format='%.16f', sep=' ')


In [10]:
v,f,d = read_vtk('/scr/ilz3/myelinconnect/struct/surf_lh/orig/mid_surface/BP4T_lh_mid.vtk')

In [21]:
sv, sf, sd = read_vtk('/scr/ilz3/myelinconnect/groupavg/indv_space/BP4T/lowres_lh_d_def.vtk')

In [23]:
sv = sv[0:10]

In [24]:
# for each vertex in the simplified mesh, find the closest vertex in the highres mesh
mapping = []
for lowres_idx in range(sv.shape[0]):
    dist = 10000
    closest = None
    for highres_idx in range(v.shape[0]):
        d = np.sqrt((v[highres_idx][0] - sv[lowres_idx][0]) ** 2 
                  + (v[highres_idx][1] - sv[lowres_idx][1]) ** 2 
                  + (v[highres_idx][2] - sv[lowres_idx][2]) ** 2)
        if d < dist:
            dist = d
            closest = highres_idx
    mapping.append([lowres_idx, closest])
mapping = np.asarray(mapping) 

In [25]:
for i in range(10):
    print sv[mapping[i][0]]
    print v[mapping[i][1]]

[ 126.39948  117.50543  139.71094]
[ 126.4      117.6      138.76149]
[ 127.66485  117.84246  139.7552 ]
[ 127.6      117.99999  138.87289]
[ 127.56905  116.22385  139.78674]
[ 127.6      116.39999  138.76111]
[ 125.91573  122.31351  138.79105]
[ 126.       122.39999  138.05463]
[ 127.56006  120.66943  139.09203]
[ 127.6      120.39999  138.72423]
[ 122.30099  120.11935  139.84627]
[ 122.       119.99999  138.89955]
[ 129.73494  119.75185  139.3521 ]
[ 129.60001  119.6      139.42366]
[ 123.22951  117.91116  139.8903 ]
[ 123.6      117.99999  138.43802]
[ 125.73241  118.87785  139.65289]
[ 125.6      118.8      138.72887]
[ 127.03365  119.28188  139.483  ]
[ 127.2      119.2      138.90459]


In [19]:
spatial.Voronoi?

In [ ]:
# for each vertex on the highres mesh find all nodes in a given radius
radius = 5
vertices = v.astype(np.float64)
faces = f.astype(np.int32)
inradius_matrix=gdist.local_gdist_matrix(vertices, faces, max_distance=radius)

In [ ]:
# for each vertex in the highres mesh find that node that corresponds to one on the lowres mesh and is closest
for highres_idx in range(v.shape[0]):
    inradius_matrix[v[highres_idx]].indices